In [8]:
import requests
import json
res = requests.get('http://sale.591.com.tw/index.php?module=search&action=rslist&is_new_list=1&type=2&searchtype=1&region=1&orderType=desc&kind=9&firstRow=0&totalRows=11734')
data = json.loads(res.text)
print data['main']


<div class="shList ">
    <ul class="shInfo">
        <li class="info">
            <div class="left" data-bind="2527012" data-img-length="13">
                <a href="sale-detail-2527012.html" class="imgbd" target="_blank" title="松菸高樓美景宅...陳偉夫"><img src="http://p2.cdn.591.com.tw/house/active/2015/08/01/143843408197208308_128x92.jpg" width="128" height="92" alt="松菸高樓美景宅...陳偉夫" /></a>
                <!-- 更多圖片 -->
                <div class="imgMore"><a href="sale-detail-2527012.html" target="_blank">13張照片</a></div>
            </div>
            <div class="right">
                <p class="title"><a href="sale-detail-2527012.html" target="_blank" class="house_url" title="松菸高樓美景宅...陳偉夫"><strong>松菸高樓美景宅...陳偉夫</strong></a><img src="./images/index/userCenter/list_vip_v1.png"  border="0" align="absmiddle" class="vipimg" alt="VIP物件" title="VIP物件"/></p>
                <p>世貿金點　台北市-信義區&nbsp;基隆路一段</p>
                <p>住宅，單價：88.98萬元，<span class="layout">1房1廳1衛</span>，</span>樓層：8/10</p>
    